In [7]:
from parlai.core.agents import create_agent
from parlai.agents.emely.emely import EmelyAgent
from parlai.core.opt import Opt
from pathlib import Path
import torch
from parlai.scripts.torchscript import export_emely
from parlai.core.agents import create_agent

# Initialize model settings

model_path = Path.cwd() / 'data/models/blender/blender_90M/'
assert model_path.is_dir()

opt_path = model_path / 'model.opt'
opt = Opt.load(opt_path)

# Change opts
opt['skip_generation'] = False
opt['init_model'] = (model_path / 'model').as_posix()
opt['no_cuda'] = True  # Cloud run doesn't offer gpu support

# Inference options
opt['inference'] = 'greedy' # 'beam' 'greedy'
opt['beam_size'] = 1

opt["scripted_model_file"] = "../../saved_models/emely_scripted_test.pt"
opt["script-module"] = "parlai.torchscript.modules:TorchScriptGreedySearch"
opt["model_file"] = opt["init_model"]

opt["temp_separator"] = "__space__"

opt["bpe_add_prefix_space"] = False

opt["input"] = "Hi! What do you like to do?"

In [8]:
# Test exporting to torchscript
original_module, scripted_module = export_emely(opt)

16:37:34 | loading dictionary from /home/ckjellson/code/emely-models/ParlAI/data/models/blender/blender_90M/model.dict
16:37:34 | num words = 54944
16:37:35 | Total parameters: 87,508,992 (87,508,992 trainable)
16:37:35 | Loading existing model params from /home/ckjellson/code/emely-models/ParlAI/data/models/blender/blender_90M/model

Generating given the original unscripted module:
 TEXT: Hi! What do you like to do?
LABEL: i like to go to the movies and play video games .

Generating given the scripted module:
 TEXT: Hi! What do you like to do?
LABEL: i like to go to the movies and play video games .


In [3]:
# Create model to test tokenization etc
emely_agent = EmelyAgent(opt)

16:29:01 | loading dictionary from /home/ckjellson/code/emely-models/ParlAI/data/models/blender/blender_90M/model.dict
16:29:02 | num words = 54944
16:29:03 | Total parameters: 87,508,992 (87,508,992 trainable)
16:29:03 | Loading existing model params from /home/ckjellson/code/emely-models/ParlAI/data/models/blender/blender_90M/model


In [4]:
text = "Hi Emely, how are you?\nI'm good thanks! What do you do for work?\nI write code and I drink coffe."
reply = emely_agent.observe_and_act(text)
print(reply)
reply = scripted_module(text)
print(reply)

that ' s cool ! what kind of code do you write ? i ' ve never written code .
that ' s cool ! what kind of code do you write ? i ' ve never written code .


In [9]:
text = "Hi Emely, how are you?"
print(emely_agent.dict.txt2vec(text))
print(scripted_module.dict.txt2vec(text))

[792, 813, 3459, 6, 102, 46, 15, 20]
[792, 813, 3459, 6, 102, 46, 15, 20]


In [12]:
# Test performance
from time import time
nruns = 1
t1 = time()
for i in range(nruns):
    reply = emely_agent.observe_and_act("Hi!\n My name is something and my friend's name is something likewise.")
    print(reply)
t2 = time()
orig_time = t2-t1
print("Time using original emely_agent: ", orig_time)
for i in range(nruns):
    reply = original_module("Hi!\n My name is something and my friend's name is something likewise.")
    print(reply)
t3 = time()
orig2_time = t3-t2
print("Time using original module: ", orig2_time)
for i in range(nruns):
    reply = scripted_module("Hi!\n My name is something and my friend's name is something likewise.")
    print(reply)
t4 = time()
script_time = t4-t3
print("Time using scripted model: ", script_time)

that ' s cool . what is your name ? i ' ve never met anyone with the same name .
Time using original emely_agent:  0.8299565315246582
that ' s cool . what is your name ? i ' ve never met anyone with the same name as me .
Time using original module:  0.7110500335693359
that ' s cool . what is your name ? i ' ve never met anyone with the same name as me .
Time using scripted model:  0.7660255432128906


In [6]:
from parlai.core.message import Message
dummy_message = Message()
dummy_message['id'] = 'localHuman'
dummy_message['text'] = 'Hi'
dummy_message['episode_done'] = False
dummy_message['label_candidates'] = None
#scripted_module.observe()

In [15]:
# Test ONNX
import torch.onnx
#torch.onnx.export(original_module, "Hi! What do you like to do?", "emely-testing.onnx")

In [8]:
print(type(emely_agent.model.encoder))

<class 'parlai.agents.transformer.modules.encoder.TransformerEncoder'>


In [9]:
print(emely_agent.dict.tokenize("Hi Emely, how are you?"))
print(scripted_module.dict.tokenize("Hi Emely, how are you?"))

['hi', 'em@@', 'ely', ',', 'how', 'are', 'you', '?']
['hi', 'em@@', 'ely', ',', 'how', 'are', 'you', '?']
